## deep HW4 Q3
## Alireza Abbasian
## 402200253

In [1]:
!pip install -U transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.0
    Uninstalling transformers-4.36.0:
      Successfully uninstalled transformers-4.36.0


In [2]:
import numpy as np

import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from torch.utils.data import random_split

import random

from transformers import AdamW

from transformers import get_linear_schedule_with_warmup

import time
import datetime
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os

from transformers import TFAutoModelForCausalLM

import re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model_name_or_path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    bos_token='<s>', 
    eos_token='</s>', 
    pad_token='<pad>',
    unk_token='<unk>'
)
tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer("<s>")["input_ids"][0], 
    eos_token_id=tokenizer("</s>")["input_ids"][0], 
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)

tokenizer.save_pretrained("/content/gpt2/")
config.save_pretrained("/content/gpt2/")

tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    "/content/gpt2",
    bos_token='<s>', 
    eos_token='</s>', 
    pad_token='<pad>'
)

print(tokenizer.encode("این کتاب است"))
print(tokenizer.encode("علیرضا عباسیان"))
print(tokenizer.encode("\n"))
print(tokenizer.encode("<s>"))
print(tokenizer.encode("</s>"))
print(tokenizer.encode("<pad>"))
print(tokenizer.encode("<|startoftext|>"))
print(tokenizer.encode("<sep>"))
print(tokenizer.encode("مخقهلاصخهدلخشهنتالخهال"))

[1002, 909, 329]
[31702, 21706]
[209]
[0]
[2]
[1]
[6]
[9]
[10537, 579, 371, 333, 944, 4485, 316, 968, 18252, 944, 324]


22 هجا بیشتر ندارد وزن شاهنامه

In [5]:
data_path = '/kaggle/input/ferdousi/ferdousi.txt'
with open(data_path, 'r', encoding='utf-8') as file:
    lines = file.read().splitlines()
    
lines = [line.replace("\n", "") for line in lines]

beyts = [lines[i] +  "<sep>" + lines[i+1] + "<sep>" for i in range(2, len(lines)-1, 2)]
two_beyt = ['<s>' + beyts[i] + "<|startoftext|>" + beyts[i+1] + '</s>' for i in range(0, len(beyts)-1, 2)]

max_length=40
encodings_dict = tokenizer(two_beyt,
                   truncation=True,
                   max_length=max_length,
                   padding="max_length")
input_ids = torch.tensor(encodings_dict['input_ids'])
attn_masks = torch.tensor(encodings_dict['attention_mask'])

In [6]:
print(two_beyt[0])

print(input_ids[0])

<s>به نام خداوند جان و خرد<sep>کزین برتر اندیشه برنگذرد<sep><|startoftext|>خداوند نام و خداوند جای<sep>خداوند روزی ده رهنمای<sep></s>
tensor([    0,   490,   561,  6733,  1305,   293,  2964,     9,  2639,   297,
         3206,  6188, 29631,   383,   343,     9,     6, 12090,   595,   561,
          293,  6733,   798,     9, 12090,   595,  3588,   546, 11552,   450,
            9,     2,     1,     1,     1,     1,     1,     1,     1,     1])


In [7]:
new_vocab = set()

for tokens in input_ids:
    new_vocab.update(tokens.tolist())

new_vocab = list(new_vocab)
print(len(new_vocab))

7648


In [8]:
class FerdousiDataset(Dataset):
    def __init__(self):
        self.input_ids = input_ids
        self.attn_masks = attn_masks

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        beyt = self.input_ids[idx]
        mask = self.attn_masks[idx]

        return beyt, mask



In [9]:
dataset =  FerdousiDataset()

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = len(dataset) - train_size - test_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("len train_dataset = ",train_dataset.__len__())
print("len test_dataset = ",test_dataset.__len__())


# Print the first three samples from the train loader

print("Train Data:")
print("**********************************")
beyt, mask =next(iter(train_loader))
for i in range(2):
    print(f"Sample {i + 1}:")
    print("token id  = ", beyt[i])
    print("mask  = ", mask[i])
    print("**********************************")

# Print the first three samples from the test loader
print("\nTest Data:")
print("**********************************")
beyt, mask = next(iter(test_loader))
for i in range(2):
    print(f"Sample {i + 1}:")
    print("token id  = ", beyt[i])
    print("mask  = ", mask[i])
    print("**********************************")

len train_dataset =  19843
len test_dataset =  2480
Train Data:
**********************************
Sample 1:
token id  =  tensor([    0, 41339,  1776,  1215, 11592, 16477,     9,   490,   314,   413,
         6482,   656, 11592, 16477,     9,     6,   799,   530, 13052,   306,
          430,   457, 17161,   330,     9,   479,  2167,   463,   303,  7838,
         3645,   330,     9,     2,     1,     1,     1,     1,     1,     1])
mask  =  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])
**********************************
Sample 2:
token id  =  tensor([    0,  8033,   367,   449,   312,   798, 36359,     9, 15035,  8803,
          664,  6176, 11114,     9,     6,   968, 21463,   596,  1481,   330,
        12798,     9, 32497,   325, 16373,  2762, 21986,     9,     2,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1])
mask  =  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [10]:
configuration = GPT2Config.from_pretrained('HooshvareLab/gpt2-fa', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("HooshvareLab/gpt2-fa", config=configuration)
model.resize_token_embeddings(len(tokenizer))



pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

Embedding(42001, 768)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model.to(device)

seed_val = 1402

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

cuda


ننظیمات زیر برای آمورش شبکه از

https://colab.research.google.com/github/hooshvare/parsgpt/blob/master/notebooks/Persian_Poetry_FineTuning.ipynb#scrollTo=uzzFo1-zIhna

الگوبرداری شده است

In [12]:
warmup_steps = 1e2
sample_every = 300
epochs = 2

from transformers import AdamW

# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters.
optimizer = AdamW(
    model.parameters(),
    lr=1e-3,
    eps=1e-8
)

total_steps = len(train_loader) * epochs


scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
def convert_token_2_beyt(token):
    beyt = tokenizer.decode(token, skip_special_tokens=False)
    beyt = beyt.replace("<|startoftext|>", "\n")
    beyt = beyt.replace("<s>", "")
    beyt = beyt.replace("</s>", "")
    beyt = beyt.replace("<pad>", "")
    beyt = beyt.replace("<sep>", "\n")
    beyt = beyt.replace('\n', ' ', 1).replace('\n', '')
    
    return beyt

In [14]:
def besoray(input_txt):
    
    model.eval()

    encodings_dict = tokenizer('<s>'  + "<|startoftext|>"+ input_txt + " " +  "</s>")
    input_id = torch.tensor(encodings_dict['input_ids'])
    mask = torch.tensor(encodings_dict['attention_mask'])
    
    input_id = input_id.unsqueeze(0)
    input_id = input_id.to(device)

    mask = mask.unsqueeze(0)
    mask = mask.to(device)

    output_id = model.generate(
        input_ids=input_id,
        do_sample=True,
        top_k=50,
        max_length=40, 
        top_p=0.95,
        num_return_sequences=1,
        attention_mask = mask
    )

    generated_beyt = convert_token_2_beyt(output_id[0])
    
    return generated_beyt

In [15]:
for epoch_i in range(0, epochs):

    print(f'epoch: {epoch_i}')

    total_train_loss = 0

    model.train()

    for step, (beyt, mask) in tqdm(enumerate(train_loader), total=len(train_loader), position=0):

        beyt = beyt.to(device)
        mask = mask.to(device)

        model.zero_grad()

        outputs = model(beyt, labels=beyt, attention_mask=mask, token_type_ids=None)

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)


    print(f'\nAverage Training Loss: {avg_train_loss}\n')
    
    model.eval()
    sample_beyt = "میازار موری که دانه کش است"
    print(f'Example input:',sample_beyt)
    gen_sample_output = besoray(sample_beyt)
    print(f'Example output: {gen_sample_output}')


    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for beyt, mask in tqdm(val_loader, total=len(val_loader), position=0):

        beyt = beyt.to(device)
        mask = mask.to(device)

        with torch.no_grad():

            outputs = model(beyt, labels=beyt, attention_mask=mask, token_type_ids=None)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(val_loader)

    print(f'\nValidation loss: {avg_val_loss}\n')

epoch: 0


100%|██████████| 621/621 [03:23<00:00,  3.06it/s]
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.



Average Training Loss: 3.55961382523443

Example input: میازار موری که دانه کش است
Example output:  میازار موری که دانه کش است به کار اندرش مغز و تن آدمیستخردمند بود از کران تا کرانهمی رای زد بر یکی سرو کوه


100%|██████████| 78/78 [00:08<00:00,  9.67it/s]



Validation loss: 2.9062254214898133

epoch: 1


100%|██████████| 621/621 [03:30<00:00,  2.95it/s]
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.



Average Training Loss: 2.517018845307846

Example input: میازار موری که دانه کش است
Example output:  میازار موری که دانه کش است بنه بر چمنها و آب و خاک استدرخت و گیا جای و بالای او


100%|██████████| 78/78 [00:08<00:00,  9.66it/s]


Validation loss: 2.662956552627759



In [16]:
model_path = 'HW4_Q3_model.pth'
torch.save(model.state_dict(), model_path)

In [45]:
print(besoray("بدو گفت رستم که که نام تو چیست"))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


 بدو گفت رستم که که نام تو چیست ازو چار چیزست و کام تو چیستکدامست با نام تو چیست


In [18]:
print(besoray("چنین گفت رستم به اسفندیار"))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


 چنین گفت رستم به اسفندیار که ای تاجور خسرو نامدارمن این پور دستان شیرگیرسرآید شما را بدین مرز دار


In [29]:
print(besoray("یکی روبهی بود بی دست"))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


 یکی روبهی بود بی دست که با جنگ ایشان نیایدت ننگبکردار دریا و دریا نمودهمان جنگشان جنگ را دست سود


In [30]:
print(besoray("به نام خداوند"))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


 به نام خداوند خداوند بخشنده و پاک تنخداوند نیکی دهش باد


In [32]:
print(besoray("که تمرین دیپ"))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


 که تمرین دیپ گرامی و راد و فرهنگ و رایبه نیروی یزدان دلیران به پای


In [44]:
print(besoray("که هنگ کنگ ما را"))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


 که هنگ کنگ ما را نه مردی بکین و نه نام و ننگنگفتی تو در چنگ مردی پلنگ
